In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import root_mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from statsmodels.tools.tools import add_constant
from sklearn.model_selection import cross_val_score, KFold
from sklearn.pipeline import make_pipeline

https://towardsdatascience.com/support-vector-regression-svr-one-of-the-most-flexible-yet-robust-prediction-algorithms-4d25fbdaca60/

In [ ]:
DROPPED = [
    "dist_360_SPEED", "dist_360_THROTTLE", "dist_360_STEER", "dist_360_BRAKE",
    "dist_360_CURRENTLAPTIMEINMS", "dist_360_LAPDISTANCE", "dist_360_WORLDPOSITIONX", "dist_360_WORLDPOSITIONY",
    "dist_360_WORLDFORWARDDIRX", "dist_360_WORLDFORWARDDIRY", "dist_360_YAW", "dist_360_PITCH",
    "dist_360_ROLL", "dist_360_left_dist", "dist_360_right_dist", "dist_360_dist_apex_1",
    "dist_360_dist_apex_2", "dist_360_angle_to_apex1", "dist_360_angle_to_apex2", "dist_360_proj_from_ref",
    "dist_430_SPEED", "dist_430_THROTTLE", "dist_430_STEER", "dist_430_BRAKE",
    "dist_430_CURRENTLAPTIMEINMS", "dist_430_LAPDISTANCE", "dist_430_WORLDPOSITIONX", "dist_430_WORLDPOSITIONY",
    "dist_430_WORLDFORWARDDIRX", "dist_430_WORLDFORWARDDIRY", "dist_430_YAW", "dist_430_PITCH",
    "dist_430_ROLL", "dist_430_left_dist", "dist_430_right_dist", "dist_430_dist_apex_1",
    "dist_430_dist_apex_2", "dist_430_angle_to_apex1", "dist_430_angle_to_apex2", "dist_430_proj_from_ref",
    "dist_530_SPEED", "dist_530_THROTTLE", "dist_530_STEER", "dist_530_BRAKE",
    "dist_530_CURRENTLAPTIMEINMS", "dist_530_LAPDISTANCE", "dist_530_WORLDPOSITIONX", "dist_530_WORLDPOSITIONY",
    "dist_530_WORLDFORWARDDIRX", "dist_530_WORLDFORWARDDIRY", "dist_530_YAW", "dist_530_PITCH",
    "dist_530_ROLL", "dist_530_left_dist", "dist_530_right_dist", "dist_530_dist_apex_1",
    "dist_530_dist_apex_2", "dist_530_angle_to_apex1", "dist_530_angle_to_apex2", "dist_530_proj_from_ref",
    "BPS_right_dist", "BPE_right_dist", "THS_right_dist", "THE_right_dist", "STS_right_dist",
    "STM_right_dist", "STE_right_dist", "APX1_right_dist", "APX2_right_dist", "BPS_CURRENTLAPTIMEINMS",
    "BPE_CURRENTLAPTIMEINMS", "THS_CURRENTLAPTIMEINMS", "THE_CURRENTLAPTIMEINMS", "STS_CURRENTLAPTIMEINMS",
    "STM_CURRENTLAPTIMEINMS", "STE_CURRENTLAPTIMEINMS", "APX1_CURRENTLAPTIMEINMS", "APX2_CURRENTLAPTIMEINMS"
]



# Uploading Data and removing outliers and features

In [37]:
data = pd.read_csv("final_data_product.csv")
data = data.dropna().drop_duplicates().drop(columns=DROPPED)
target_mean = data["Target_CURRENTLAPTIMEINMS"].mean()
target_std = data["Target_CURRENTLAPTIMEINMS"].std()
data = data[data['Target_CURRENTLAPTIMEINMS'] < target_mean + 3 * target_std] # removes 12 longest times
y = data["Target_CURRENTLAPTIMEINMS"]
X = data.drop(columns=["Target_CURRENTLAPTIMEINMS", "lap_id", "invalid_lap"])

target_columns = [
    'target_CURRENTLAPTIMEINMS', '_LAPDISTANCE', '_WORLDPOSITIONX', 
    '_WORLDPOSITIONY', '_STEER', '_BRAKE', '_THROTTLE', '_SPEED',
]

selected_columns = [col for col in X.columns if col.endswith(tuple(target_columns))]

X = X[selected_columns]

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=42)

scaler_X = StandardScaler()
scaler_X_split = StandardScaler()
scaler_y = StandardScaler()
scaler_y_split = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
X_train_scaled = scaler_X_split.fit_transform(X_train)
X_test_scaled = scaler_X_split.transform(X_test)

y_train_scaled = scaler_y_split.fit_transform(y_train.to_numpy().reshape(-1, 1)).ravel()
y_test_scaled = scaler_y_split.transform(y_test.to_numpy().reshape(-1, 1)).ravel()
y_scaled = scaler_y.fit_transform(y.to_numpy().reshape(-1, 1)).ravel()

# Feature selection

### Mutual information

In [ ]:
from sklearn.feature_selection import mutual_info_regression

m_info = mutual_info_regression(X, y)
Scores = pd.DataFrame(sorted(zip(X.columns, m_info), key=lambda x: x[1], reverse=True), columns=["feature", "mi_score"])
pd.set_option('display.max_rows', 200)
Scores

# Detecting mutlicollinearity

### Variance inflation factor

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_data = pd.DataFrame()
X = add_constant(X)
vif_data['feature'] = X.columns
vif_data["VIF"] = [round(variance_inflation_factor(X.values, i), 4) for i in range(X.shape[1])]
vif_data[vif_data["VIF"] < 10].sort_values(by="VIF")#.iloc[:,0]
# vif_data

# Modelling

### SVR (scaled data without feature selection and without addressing mutlicollinearity)

In [ ]:
poly_parameters = {
    'kernel': ['poly'],
    'degree': [2, 3, 4, 5, 6, 7, 8],
    'gamma': ['scale', 'auto'],
    'coef0': [1, 3, 5],
    'tol': [1e-6],
    'C': [0.3, 0.5, 1, 10],
    'epsilon': [0.0005],
    'shrinking': [True],
    'verbose': [True],
    'max_iter': [-1]
}

from sklearn.svm import SVR

grid_poly = GridSearchCV(
    estimator=SVR(),
    param_grid=poly_parameters,
    cv=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=2
)

grid_poly.fit(X_train_scaled, y_train_scaled)
print("Best parameters:", grid_poly.best_params_)
print("Best RMSE:", abs(grid_poly.best_score_))
y_pred_poly_scaled = grid_poly.predict(X_test_scaled)
y_pred_poly = scaler_y_split.inverse_transform(y_pred_poly_scaled.reshape(-1,1))

rmse_poly = root_mean_squared_error(y_pred_poly, y_test)
r2_poly = r2_score(y_pred_poly, y_test)
print("RMSE for poly kernel:", rmse_poly)
print("R² for poly kernel:", r2_poly)

Fitting 5 folds for each of 168 candidates, totalling 840 fits
[LibSVM]Best parameters: {'C': 0.3, 'coef0': 1, 'degree': 2, 'epsilon': 0.0005, 'gamma': 'auto', 'kernel': 'poly', 'max_iter': -1, 'shrinking': True, 'tol': 1e-06, 'verbose': True}
Best RMSE: 0.5267111780051136
RMSE for poly kernel: 1411.7489241080818
R² for poly kernel: 0.7493263530455132


Fitting 5 folds for each of 30 candidates, totalling 150 fits
[LibSVM]Best parameters: {'C': 0.3, 'coef0': 3, 'degree': 3, 'epsilon': 0.0005, 'gamma': 'scale', 'kernel': 'poly', 'max_iter': -1, 'shrinking': True, 'tol': 1e-06, 'verbose': True}
Best RMSE: 0.549755590951283
RMSE for poly kernel: 1551.693230307914
R² for poly kernel: 0.6867939663481857

<!-- Fitting 5 folds for each of 30 candidates, totalling 150 fits
[LibSVM]Best parameters: {'C': 0.3, 'coef0': 3, 'degree': 3, 'epsilon': 0.0005, 'gamma': 'scale', 'kernel': 'poly', 'max_iter': -1, 'shrinking': True, 'tol': 1e-06, 'verbose': True}
Best RMSE: 0.549755590951283
RMSE for poly kernel: 1551.693230307914
R² for poly kernel: 0.6867939663481857 -->

### CV

In [27]:
fold_rmse = []
fold_R2 = []

for i in range(4):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.25, random_state=i+4)
    model = SVR(kernel='poly', degree=3, gamma='scale',
                    coef0=3, tol=1e-3,C=0.1, epsilon=0.0005,
                    shrinking=True, verbose=2,max_iter=-1)
    
    scaler_X_split = StandardScaler()
    scaler_y_split = StandardScaler()
    X_train_scaled = scaler_X_split.fit_transform(X_train)
    X_test_scaled = scaler_X_split.transform(X_test)
    y_train_scaled = scaler_y_split.fit_transform(y_train.to_numpy().reshape(-1, 1)).ravel()
    y_test_scaled = scaler_y_split.transform(y_test.to_numpy().reshape(-1, 1)).ravel()

    model.fit(X_train_scaled, y_train_scaled)

    y_pred_poly_scaled = model.predict(X_test_scaled)
    y_pred_poly = scaler_y_split.inverse_transform(y_pred_poly_scaled.reshape(-1,1))

    fold_rmse.append(root_mean_squared_error(y_pred_poly, y_test))
    fold_R2.append(r2_score(y_pred_poly, y_test))

print("\n4-fold CV RMSE:")
print("Fold RMSEs:", np.round(fold_rmse, 3))
print("Mean RMSE :", np.round(np.mean(fold_rmse), 3))
print("Std  RMSE :", np.round(np.std(fold_rmse), 3))
print("\n4-fold CV R2:")
print("Fold RMSEs:", np.round(fold_R2, 3))
print("Mean RMSE :", np.round(np.mean(fold_R2), 3))
print("Std  RMSE :", np.round(np.std(fold_R2), 3))

[LibSVM][LibSVM][LibSVM][LibSVM]
4-fold CV RMSE:
Fold RMSEs: [1279.737 1603.639 1805.64  2243.22 ]
Mean RMSE : 1733.059
Std  RMSE : 349.207

4-fold CV R2:
Fold RMSEs: [0.835 0.543 0.727 0.591]
Mean RMSE : 0.674
Std  RMSE : 0.115


mean rmse 1733

In [40]:
rbf_parameters = {
    'kernel': ['rbf'],
    'gamma': ['scale', 'auto'],
    'tol': [1e-3],
    'C': [1, 1.2, 2],
    'epsilon': [0.001],
    'shrinking': [True],
    'verbose': [False],
    'max_iter': [-1]
}

grid_rbf = GridSearchCV(
    estimator=SVR(),
    param_grid=rbf_parameters,
    cv=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=3
)

grid_rbf.fit(X_train_scaled, y_train_scaled)
print("Best parameters:", grid_rbf.best_params_)
print("Best RMSE:", abs(grid_rbf.best_score_))
y_pred_rbf_scaled = grid_rbf.predict(X_test_scaled)
y_pred_rbf = scaler_y_split.inverse_transform(y_pred_rbf_scaled.reshape(-1,1))

rmse_rbf = root_mean_squared_error(y_pred_rbf, y_test)
r2_rbf = r2_score(y_pred_rbf, y_test)
print("RMSE for rbf kernel:", rmse_rbf)
print("R² for rbf kernel:", r2_rbf)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Best parameters: {'C': 2, 'epsilon': 0.001, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': -1, 'shrinking': True, 'tol': 0.001, 'verbose': False}
Best RMSE: 0.6609561496644532
RMSE for rbf kernel: 2170.7917144779694
R² for rbf kernel: -0.39805540023762354


# Finding optimum using model

In [55]:
# --- 7) Define a realistic search box (stay inside observed data) ---
percentiles = (0.05, 0.95)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)
y_scaled_df = pd.DataFrame(y_scaled, columns=["Target_CURRENTLAPTIMEINMS"])

bounds = {
    f: (X_scaled_df[f].quantile(percentiles[0]), X_scaled_df[f].quantile(percentiles[1]))
    for f in X_scaled_df.columns
}

# --- 8) Random search over SVR prediction surface to find min predicted target ---
rng = np.random.default_rng(42)
N = 50_000
candidates = {
    f: rng.uniform(low=b[0], high=b[1], size=N)
    for f, b in bounds.items()
}
Xcand = pd.DataFrame(candidates)[X_scaled_df.columns]

# Predict on scaled values
ycand = grid_poly.predict(Xcand)

# Inverse-transform both features and target
ycand_unscaled = scaler_y.inverse_transform(ycand.reshape(-1, 1)).ravel()

imin = int(np.argmin(ycand_unscaled))
best_combo_scaled = Xcand.iloc[imin].to_frame().T  # one-row DataFrame
best_combo_unscaled = pd.DataFrame(
    scaler_X.inverse_transform(best_combo_scaled),
    columns=X.columns
)

best_pred = ycand_unscaled[imin]

print("\n=== SVR-suggested first-brake setup (within observed range) ===")
for k, v in best_combo_unscaled.iloc[0].items():
    print(f"{k}: {v:,.4f}")
print(f"Predicted Target_CURRENTLAPTIMEINMS: {best_pred:,.3f}")


c:\Users\Haseeb Ijaz\AppData\Local\Programs\Python\Python314\Lib\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but SVR was fitted without feature names
  warnings.warn(



=== SVR-suggested first-brake setup (within observed range) ===
BPS_SPEED: 316.8764
BPS_THROTTLE: 0.0259
BPS_STEER: -0.0086
BPS_BRAKE: 0.0000
BPS_LAPDISTANCE: 289.9898
BPS_WORLDPOSITIONX: 249.7089
BPS_WORLDPOSITIONY: 252.2799
BPS_ext_LAPDISTANCE: 319.7515
BPE_SPEED: 284.9766
BPE_THROTTLE: 0.0859
BPE_STEER: 0.1857
BPE_BRAKE: 0.7587
BPE_LAPDISTANCE: 308.9894
BPE_WORLDPOSITIONX: 364.2086
BPE_WORLDPOSITIONY: 270.1854
BPE_ext_LAPDISTANCE: 380.7259
THS_SPEED: 310.8271
THS_THROTTLE: 0.5563
THS_STEER: -0.0149
THS_BRAKE: 0.1010
THS_LAPDISTANCE: 198.4681
THS_WORLDPOSITIONX: 288.2920
THS_WORLDPOSITIONY: 321.3616
THS_ext_LAPDISTANCE: 309.7240
THE_SPEED: 173.9855
THE_THROTTLE: 0.2880
THE_STEER: 0.3508
THE_BRAKE: 0.4633
THE_LAPDISTANCE: 373.8134
THE_WORLDPOSITIONX: 361.8980
THE_WORLDPOSITIONY: 177.8730
THE_ext_LAPDISTANCE: 484.3836
STS_SPEED: 170.5146
STS_THROTTLE: 0.6255
STS_STEER: 0.0073
STS_BRAKE: 0.1512
STS_LAPDISTANCE: 361.3141
STS_WORLDPOSITIONX: 359.7168
STS_WORLDPOSITIONY: 210.7334
STS_ext_

# Partial Poly Graph

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler

# Example: 3 feature dataset (X1, X2, X3)
# X is your input matrix with shape (n_samples, n_features)
# y is your target variable
X = np.random.rand(100, 3) * 10  # Random data with 3 features
y = np.sin(X[:, 0]) + 0.5 * X[:, 1] + np.random.randn(100)  # Target variable with noise

# Scaling the data
sc_X = StandardScaler()
sc_y = StandardScaler()

X_scaled = sc_X.fit_transform(X)
y_scaled = sc_y.fit_transform(y.reshape(-1, 1)).ravel()

# Train the SVR model with a polynomial kernel
svr_poly = SVR(kernel='poly', degree=3, C=100, epsilon=0.1)
svr_poly.fit(X_scaled, y_scaled)

# 1. Compute the 95th percentiles for each feature (for input ranges)
percentiles = np.percentile(X, 95, axis=0)

# 2. Plot Partial Poly Graphs for each feature
for feature_index in range(X.shape[1]):
    # Vary the current feature (e.g., X1) across the top 95 percentile range
    feature_range = np.linspace(0, percentiles[feature_index], 1000).reshape(-1, 1)

    # Fix the other features at their mean values
    X_fixed = np.mean(X[:, np.arange(X.shape[1]) != feature_index], axis=0)

    # Create the combined input data with the varied feature and fixed others
    X_combined = np.hstack([feature_range, np.tile(X_fixed, (feature_range.shape[0], 1))])

    # Scale the new data for prediction
    X_combined_scaled = sc_X.transform(X_combined)

    # Predict using the trained SVR model
    y_pred_scaled = svr_poly.predict(X_combined_scaled)

    # Inverse transform the predictions to get original scale
    y_pred = sc_y.inverse_transform(y_pred_scaled)

    # Plot the result
    plt.figure(figsize=(8, 6))
    plt.plot(feature_range, y_pred, label=f"SVR Polynomial Fit (Varying Feature {feature_index+1})")
    plt.title(f'Partial Poly Graph (Varying Feature {feature_index+1})')
    plt.xlabel(f'Feature {feature_index+1}')
    plt.ylabel('Predicted y')
    plt.legend()
    plt.show()

# 3. Plot Partial Dependence for Pairs of Features (2D Contour Plots)
for i in range(X.shape[1]):
    for j in range(i + 1, X.shape[1]):
        # Create a grid of values for Feature i and Feature j
        feature1_range = np.linspace(0, percentiles[i], 100)
        feature2_range = np.linspace(0, percentiles[j], 100)
        
        # Create a meshgrid for the 2D grid of Feature i and Feature j
        feature1_grid, feature2_grid = np.meshgrid(feature1_range, feature2_range)
        
        # Create a combined grid of features for prediction
        X_grid = np.vstack([feature1_grid.ravel(), feature2_grid.ravel()]).T
        
        # Fix the other features at their mean value
        X_fixed = np.mean(X[:, [k for k in range(X.shape[1]) if k != i and k != j]], axis=0)
        
        # Combine the grid values with the fixed feature values
        X_combined = np.hstack([X_grid, np.full((X_grid.shape[0], len(X_fixed)), X_fixed)])

        # Scale the new data for prediction
        X_combined_scaled = sc_X.transform(X_combined)

        # Predict using the trained SVR model
        y_pred_scaled = svr_poly.predict(X_combined_scaled)

        # Inverse transform the predictions to get original scale
        y_pred = sc_y.inverse_transform(y_pred_scaled)

        # Reshape the predictions back into the grid shape
        y_pred_grid = y_pred.reshape(feature1_grid.shape)

        # Plot the results as a contour plot (for 2D data)
        plt.figure(figsize=(8, 6))
        plt.contourf(feature1_range, feature2_range, y_pred_grid, levels=50, cmap='coolwarm')
        plt.colorbar(label='Predicted y')
        plt.title(f'Partial Dependence (Varying Features {i+1} and {j+1})')
        plt.xlabel(f'Feature {i+1}')
        plt.ylabel(f'Feature {j+1}')
        plt.show()


In [ ]:
# poly_parameters = {
#     'kernel': ['poly'],
#     'degree': [3, 5, 7, 9],
#     'gamma': ['scale', 'auto'],
#     'coef0': [0, 1, 3],
#     'tol': [1e-3],
#     'C': [0.01, 0.1, 1],
#     'epsilon': [0.01, 0.05, 0.1, 1],
#     'shrinking': [True],
#     'verbose': [False],
#     'max_iter': [-1]
# }


# rbf_parameters = {
#     'kernel': ['rbf'],
#     'gamma': ['scale', 'auto'],
#     'tol': [1e-3],
#     'C': [0.01, 0.1, 1],
#     'epsilon': [0.01, 0.05, 0.1, 0.5, 1],
#     'shrinking': [True],
#     'verbose': [False],
#     'max_iter': [-1]
# }
# grid_poly = GridSearchCV(
#     estimator=SVR(),
#     param_grid=poly_parameters,
#     cv=5,
#     scoring='neg_root_mean_squared_error', # check others
#     n_jobs=-1,
#     verbose=2
# )

# grid_poly.fit(X_train_scaled, y_train)
# print("Best parameters:", grid_poly.best_params_)
# print("Best RMSE:", abs(grid_poly.best_score_))

# grid_rbf = GridSearchCV(
#     estimator=SVR(),
#     param_grid=rbf_parameters,
#     cv=5,
#     scoring='neg_root_mean_squared_error', # check others
#     n_jobs=-1,
#     verbose=3
# ) 

# grid_rbf.fit(X_train_scaled, y_train)
# print("Best parameters:", grid_rbf.best_params_)
# print("Best RMSE:", abs(grid_rbf.best_score_))

In [ ]:
rbf_parameters = {
    'kernel': ['rbf'],
    'gamma': ['scale', 'auto'],
    'tol': [1e-3],
    'C': [0.01, 0.1, 1],
    'epsilon': [0.01, 0.05, 0.1, 0.5, 1],
    'shrinking': [True],
    'verbose': [False],
    'max_iter': [-1]
}

grid_rbf = GridSearchCV(
    estimator=SVR(),
    param_grid=rbf_parameters,
    cv=5,
    scoring='neg_root_mean_squared_error', # check others
    n_jobs=-1,
    verbose=3
) 
